# Optimización matemática

La **optimización** es el procedimiento para:

> Encontrar la mejor solución para un problema dentro de un conjunto de posibilidades

La optimización es un área bastante estudiada de las matemáticas y algunos problemas de optimización requieren de soluciones muy específicas

El objetivo de esta lección es entregar una revisión general a los problemas de optimización que podemos resolver usando las herramientas del módulo `scipy.optimize`

In [19]:
%matplotlib inline
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

## Problema general de optimización

Usualmente un problema matemático de optimización se formula como

$$
\begin{align}
\min_x &f(x) \nonumber \\
\text{sujeto a: } & g_i(x) = 0, i=1,2\ldots, I \\ \nonumber 
& h_j(x) \leq 0, j=1,2,\ldots J \nonumber
\end{align}
$$

donde 

- $x \in \mathbb{R}^D$ se conoce como **variable o variables de decisión**
- $f : \mathbb{R}^D \to \mathbb{R}$ se conoce como **función objetivo**
- $g_i : \mathbb{R}^D \to \mathbb{R}$ se conocen como **restricciones de igualdad** 
- $h_j : \mathbb{R} \to \mathbb{R}^H$ se conocen como **restricciones de desigualdad**

El problema de optimización es entonces

> La búsqueda de un valor extremo de la función objetivo dentro del espacio definido por las restricciones

Un valor extremo puede ser un mínimo o un máximo. En un problema particular usualmente sólo nos interesa uno de estos casos. 

Dado que

$$
\max_x f(\vec x) \equiv \min_x - f(\vec x),
$$

entonces hablaremos sólo de minimización sin pérdida de generalidad

### Reconocer y clasificar problemas de optimización

Estudiando algunas características del problema podemos seleccionar más fácilmente un algoritmo  apropiado para resolverlo. Algunas preguntas guía que podemos realizar son

¿Es mi función objetivo de una variable ($D=1$) versus multi-variable ($D>1$)?

> Esto define la dimensionalidad o escala del problema


¿Existen restricciones de igualidad y/o desigualidad que debo cumplir?

> Algunos algoritmos sólo pueden resolver problemas sin restricciones

¿Es mi función objetivo lineal o no lineal con respecto a la entrada?

> Si todas las funciones son lineales entonces se pueden usar técnicas de **programación lineal**. Esto problemas son más simples que los no lineales

¿Es mi función objetivo convexa o no convexa?

> Una función no-convexa (derecha) puede tener múltiples mínimos locales. Por el contrario una función convexa (izquierda) tiene un único mínimo

<img src="img/opti1.png">

¿Es mi función objetivo continua y diferenciable o no-diferenciable?

> Muchos métodos se basan en el gradiente de la función de costo para encontrar la solución óptima. Si la función objetivo no es suave y no puede diferenciarse entonces no podemos usar dichos métodos


 


## Resolviendo un problema de optimización

Consideremos primero el caso de una **función objetivo continua y derivable**. 

Veremos primero una solución analítica, luego una solución exhaustiva y finalmente una solución basada en métodos iterativos


### Solución analítica

La forma más clásica para obtener la solución en este caso es encontrar las raices (ceros) de la derivada/gradiente de $f$. Es decir

$$
\nabla f (x^*) = \begin{pmatrix} \frac{\partial f}{\partial x_1}, \frac{\partial f}{\partial x_2}, \ldots, \frac{\partial f}{\partial x_D} \end{pmatrix} = \vec 0
$$

Estas soluciones se conocen como **puntos estacionarios** de $f$, que incluyen los mínimos, máximos y puntos silla

Luego si las segunda derivada o matriz Hessiana de $f$

$$
H_{ij}^f (x)  = \frac{\partial^2 f}{\partial x_i \partial x_j} (x^*)
$$

es positiva o semi-definida positiva entonces $x^*$ es un **mínimo local**

**Receta**

1. Obtener $x^*$ tal que $\nabla f (x^*)=0$
1. Probar que es un mínimo el Hessiano

**Limitación** 

Sólo es práctico si podemos despejar una expresión análitica de $x$ a partir de $\nabla f (x^*)=0$. Esto se puede hacer algebraicamente o usando una librería de cálculo simbólico como [SimPy](https://www.sympy.org/en/index.html) o [jax](https://github.com/google/jax)

**Ejemplos**

Consideremos el siguiente problema

$$
\min_x x^2 - 2x
$$

Igualando la primera derivada de la función objectivo a cero tenemos que $ 2x - 2 = 0$, es decir $x=1$ es un punto estacionario

La segunda derivada es mayor que cero por lo tanto corresponde a un mínimo

Considere ahora el siguiente problema no-convexo

$$
f(x) = x^2 - 2x + 5 \sin(2x)
$$

La derivada en este caso es

$$
\frac{\partial f}{\partial x} = 2x - 2 + 10 \cos(2x) = 0
$$

En este caso no es posible despejar analiticamente $x$ 

### Búsqueda exhaustiva de la mejor solución

Podemos encontrar la mejor solución probando una gran cantidad de "soluciones candidatas" de forma numérica y guardando la mejor. Esto se suele describir como un "método de fuerza bruta". 

**Receta**

1. Definimos una grilla para nuestro espacio de parámetros (dominio y resolución)
1. Para cada elemento de la grilla calculamos la función de costo
1. Buscamos el elemento con menor función de costo

**Ventaja** 

Si la resolución es lo suficientemente fina podemos encontrar el mínimo global del dominio aunque la función sea no-convexa

**Desventaja** 

El costo computacional crece rapidamente con la dimensión de $x$, **explosión combinatorial**. Esto lo hace infactible en la mayoría de los casos reales

### Método iterativos

En lugar de evaluar todo el espacio de posibilidades los métodos iterativos parten de una solución inicial y la refinan paso a paso. 

En cada paso los métodos iterativos buscan la dirección que más los acerque a la solución óptima

A continuación veremos algunos métodos iterativos clásicos para funciones continuas y derivables


**Método de Newton**

Sea el valor actual de la variable de decisión $x_t$. Podemos escribir el valor que tendrá en el siguiente paso como

$$
x_{t+1} = x_t + \Delta x
$$

Lo que queremos es encontrar el mejor $\Delta x$ según nuestra función objetivo. 

Consideremos la aproximación de Taylor de segundo orden de $f$

$$
f(x_{t} + \Delta x) \approx f(x_t) + \nabla f (x_t) \Delta x + \frac{1}{2} \Delta x^T H_f (x_t) \Delta x 
$$

Si derivamos en función de $\Delta x$ e igualamos a cero se tiene que

$$
\begin{align}
\nabla f (x_t)  +  H_f (x_t) \Delta x &= 0 \nonumber \\
\Delta x &= - [H_f (x_t)]^{-1}\nabla f (x_t)  \nonumber \\
x_{t+1} &= x_{t} - [H_f (x_t)]^{-1}\nabla f (x_t)  \nonumber 
\end{align}
$$

Que corresponde a la regla iterativa de Newton. La regla está función del **Gradiente** y del **Hessiano** de $f$

Notar que

- La solución depende de $x_0$ el valor inicial
- Al utilizar el método de Newton estamos suponiendo que la aproximación de segundo orden es suficiente para nuestro problema
- Si nuestro modelo tiene $M$ parámetros el Hessiano será una matriz de $M\times M$. Si $M$ es muy grande usar el Hessiano podría ser infactible

**Gradiente descendente (GD)**

Si el Hessiano es prohibitivo podemos usar una aproximación de primer orden de la regla de Newton. Esto resulta en el clásico método conocido como  **gradiente descendente**

$$
x_{t+1} = x_{t} - \eta \nabla f (x_t)
$$

donde se reemplaza el Hessiano por una constante $\eta$ llamado "paso" o "tasa de aprendizaje". 

Es sumamente importante calibrar adecuadamente este parámetro. A continuación veremos un ejemplo para ilustrar los comportamientos que del gradiente descedente ante distintas tasas de aprendizaje.

Luego veremos que ocurre cuando optimizamos una función no convexa usando GD (o método de Newton)



**Ejemplo** Influencia de la tasa de aprendizaje en GD

¿Cómo cambia la optimización con distintos $\eta$?

- Un $\eta$ muy pequeño hará que la convergencia sea muy lenta
- Un $\eta$ muy grande hará que la optimización sea inestable o que diverja

Consideremos nuevamente optimizar la función 

$$
f(x) = x^2 - 2x
$$

esta vez utilizando gradiente descedente

In [17]:
%%capture 
fig, ax = plt.subplots(1, 3, figsize=(7, 2.5), tight_layout=True, sharey=True)

x_plot = np.linspace(-4, 6, num=100)
f = lambda x : x**2 - 2*x
df = lambda x : 2*x - 2
x = [np.random.rand(5)*10-4 for k in range(3)]
etas = [0.011, 0.11, 1.1]
sc = []

for k in range(3):
    ax[k].plot(x_plot, f(x_plot))
    sc.append(ax[k].scatter(x[k], f(x[k]), c='k', s=100))
    ax[k].set_ylabel(r'$f(x)$')
    ax[k].set_xlabel(r'$x$')
    ax[k].set_title(f'eta={etas[k]}')

def update_plot(n):
    for k in range(3):
        x_ = sc[k].get_offsets()[:, 0]
        x_ -= etas[k]*df(x_)
        sc[k].set_offsets(np.c_[x_, f(x_)])
    return sc[0], sc[1], sc[2]
    
anim = animation.FuncAnimation(fig, update_plot, frames=20, interval=200, repeat=False, blit=True)

En el siguiente ejemplo animado

- Cada punto negro es una solución que parte de un valor inicial distinto
- La linea azul es la función objetivo 
- Cada figura representa una tasa de aprendizaje distinta

In [20]:
HTML(anim.to_html5_video())

Luego de 20 iteraciones podemos ver que

- En el caso de la derecha las soluciones divergen
- En el caso de la izquierda las soluciones no alcanzan a converger
- En el caso central las soluciones convergen

**Ejemplo:** Optimización de una función no convexa con GD

Consideremos la función no convexa

$$
f(x) = x^2 - 2x + 5 \sin(2x)
$$

la cual optimizaremos usando gradiente descedente. Recordemos que GD sólo puede garantizar que la solución es un punto estacionario. 

Si la función es no convexa entonces la solución dependerá fuertemente del valor inicial de $x$

In [21]:
%%capture 

x_plot = np.linspace(-4, 6, num=100)
f = lambda x : x**2 - 2*x + 5*np.sin(2*x)
df = lambda x : 2*x - 2 + 10*np.cos(2*x)
x = np.linspace(-4, 6, num=10)
eta = 0.005

fig = plt.figure(figsize=(7, 4), tight_layout=True)
gs = fig.add_gridspec(3, 3)
ax1 = fig.add_subplot(gs[0:2, :])
ax2 = fig.add_subplot(gs[2, :])

ax1.plot(x_plot, f(x_plot), label=r'$x^2-2x+5\sin(2x)$')
ax2.plot(x_plot, -df(x_plot))
ax2.plot(x_plot, [0]*len(x_plot), 'r--')
sc = ax1.scatter(x, f(x), s=100, c='k', label='soluciones')

ax1.set_ylabel(r'$f(x)$')
ax1.legend()
ax2.set_xlabel(r'$x$')
ax2.set_ylabel(r'$-\nabla f(x)$')

def update_plot(n):
    ax1.set_title(f"Iteración {n}/50")
    x = sc.get_offsets()[:, 0]
    x -= eta*df(x)
    sc.set_offsets(np.c_[x, f(x)])
    return sc,
    
anim = animation.FuncAnimation(fig, update_plot, frames=50, interval=200, repeat=False, blit=True)

En el siguiente ejemplo animado

- Cada punto negro es una solución que parte de un valor inicial distinto
- El gráfico superior es la función objetivo 
- El gráfico inferior es el gradiente de al función objetivo y la linea punteada roja son los ceros de la derivada


In [22]:
HTML(anim.to_html5_video())

Luego de 50 iteraciones podemos ver como distintas soluciones iniciales resultan en distintos mínimos

Una estrategia cuando se usa GD en funciones no-convexas es justamente utilizar y comparar varias condiciones iniciales distintas


## Módulo [`scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#optimization-scipy-optimize)

Podemos realizar optimización numérica usando el módulo `scipy.optimize`. La función principal de este módulo es `minimize` la cual engloba a una larga lista de optimizadores

Sus argumentos principales son

```python
from scipy.optimize import minimize
minimize(fun, # Función objetivo 
         x0, # Valor inicial de la variable de decisión
         args=(), # Argumentos adicionales de fun
         method=None, # El método de optimización a usar (más detalles a continuación)
         jac=None, # Función que calcula la matriz de primeras derivadas (jacobiano)
         bounds=None, # Secuencia de tuplas (min, max) con cotas para x 
         constraints=(), # Diccinario o lista de restricciones (más detalles a continuación)
         tol=None, # Tolerancia para el término de la optimización
         callback=None, # Una función que se ejecuta luego de cada iteración
         options=None, # Diccionario con las opciones especificas para cada método
         ...
        )
```

La función objetivo debe estar definida de la siguiente forma

```python
def fun(x, *args):
    ...
    return f 
```

donde `x` es la variable a optimizar y `f` debe ser un valor escalar flotante. Los argumentos adicionales a `x` se deben desempaquetar de la tupla `args`

La función de primeras derivadas debe seguir una forma similar

```python
def jac(x, *args):
    ...
    return dx # Esto es un arreglo con la misma dimesión de x
```

donde `x` y `args` deben ser coincidir con `fun`. El argumento `jac` es opcional. Si no se especifica las derivadas se calcularán de forma numérica, lo cual es menos eficiente. Además recordar que no todos los métodos requieren de primeras derivadas

La función `optimize` retorna un objeto de tipo [`OptimizeResult`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html#scipy.optimize.OptimizeResult), cuyos atributos más importantes son

- `x`: Mejor valor encontrado de la variable de decisión
- `fun`: Valor de la función objetivo en el óptimo encontrado
- `jac`: Valor de la matriz de primeras derivadas en el óptimo encontrado
- `success`: Booleano que indica si la optimización se llevó a cabo con exito
- `message:` Mensaje indicando la razón de término, útil para debuggear

A continuación describiremos algunos de los métodos disponibles a través del argumento `method` de `minimize`

### Optimización sin restricciones 

Con estos métodos no se puede especificar el argumento `constraint` o `bounds`. Por ejemplo están

- [`method=CG`](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-cg.html#optimize-minimize-cg): Gradiente conjugado. Es una versión de GD con tasa de aprendizaje adaptiva
- [`method=BFGS`](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-bfgs.html#optimize-minimize-bfgs): Es un método de tipo [quasi-Newton](https://en.wikipedia.org/wiki/Quasi-Newton_method) con Hessiano inverso aproximado a cada paso. Es el método por defecto en optimize y es en general una buena opción

Los cuales usan gradientes, ya sea numérico o especificado mediante el argumento `jac`. Si la derivada puede obtenerse analiticamente y es confiable los siguientes métodos tendrán un desempeño superior a las alternativas
    
Luego están

- [`method=Nelder-Mead`](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-neldermead.html#optimize-minimize-neldermead): Es una heurística tipo simplex. [Animación que muestra su funcionamiento](https://www.youtube.com/watch?v=HUqLxHfxWqU)
- [`method=Powell`](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-powell.html#optimize-minimize-powell): Algoritmo de búsqueda de linea siguiendo una dirección a la vez. [Animación que muestra su funcionamiento](https://www.youtube.com/watch?v=4TYJGihyuDg)

Los cuales no usan gradientes. Estos métodos pueden usarse cuando la función objetivo es no-derivable o demasiado ruidosa para ser derivada

### Optimización con restricciones

Con estos métodos se pueden incorporar restricciones al problema ya sea en forma de cotas para las variables o ecuaciones de igualdad/desigualdad que las variables deben cumplir

- Las restricciones de igualdad deben ser siempre de la forma $g(x) = 0$
- Las restricciones de desigualdad deben ser siempre de la forma $h(x) \geq 0 $

En la práctica las restricciones se entregan como una tupla en el argumento `constraint` de `method`. Cada restricción es un diccionario con las llaves `type` y `fun` para especificar el tipo (string `eq` o `ineq`) y la función, respectivamente. Opcionalmente se puede especificar `jac`, la matriz de primeras derivadas de `fun` y `arg` una tupla con argumentos adicionales para `fun` y `jac`

Por ejemplo si tengo la siguiente restricción (de desigualdad)

$$
x^2 \geq 1 + 2x
$$

la tengo que escribir como:

```python
>>> h1 = {'type': 'ineq', 
          'fun' : lambda x: x**2 - 2*x -1,
          'jac' : lambda x: np.array([2*x - 2])}
```

Los métodos que permiten especificar restricciones son

- `method=L-BFGS-B`: Similar a BFGS pero permite añadir cotas para la variable de decisión
- `method=SLSQP`: *Sequential Least Squares Programming*. Este método acepta cotas, restricciones de igualdad y restricciones de desigualdad

Sea la siguiente función de costo con dos variables de decisión

$$
\min f(x, y) = -(2xy+2x-x^2-2y^2) 
$$

sujeta a 

$$
x^3 - y = 0 ~\wedge~y-(x-1)^4-2 \geq 0 
$$

donde

$$
0.5\leq x \leq 1.5 ~\wedge~ 1.5 \leq y \leq 2.5
$$

- Escriba la función de costo, restricciones y cotas
- Muestre la solución del problema de optimización obtenida con BFGS (ignorando restricciones y cotas), L-BFGS-B (ignorando restricciones) y SLSQP. Use $x_0 = 0$ y $y_0 = 1$ como solución inicial
- (Opcional) Muestre graficamente la solución del problema

**Solución paso a paso con comentarios**

In [ ]:
YouTubeVideo_formato('60nw7S7eo8c')

<img src="img/opti2.png">

En la gráfica
- El gradiente de color es la función objetivo
- La sombra rectangular son las cotas
- La linea punteada es la restricción de igualdad
- La linea de puntos es la restricción de desigualdad

:::{seealso}

Si quieres profundizar en este tema sugiero revisar

- [Comparativa detallada entre los distintos métodos de optimización](https://scipy-lectures.org/advanced/mathematical_optimization/index.html) 
- [Encontrando raices de una función con `scipy`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html#scipy.optimize.root)
- [Problemas de programación lineal con `scipy`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html#scipy.optimize.linprog)

:::